# import

In [1]:
from utils.mapping import (
    separate_text,
    separate_word,
    normalize_punctuation_spacing,
    contraction,
    word_tokenize_vn,
)

from utils.statistic import (
    draw_graph,
    draw_hist_graph,
    get_length_tokens,
)

from utils.save import (
    write_file,
)

from utils.filter import (
    condition_length_with_tokenizer,
    condition_non_number_character,
)

from utils.tokenizers import (
    ApiTokenizerHuggingFace,
    read_tokenizer,
    BPE_TOKEN,
    WORDPIECE_TOKEN,
    WORDLEVEL_TOKEN,
)

from utils.seed import set_seed

import pandas as pd
import torch

# seed

In [2]:
set_seed(seed=42)

# config

In [3]:
config = {}


config["max_sample"] = 600000
config["max_get_sample"] = 2000000

config["raw_data_path"] = "./dataset/bart/pretrain/raw_data/dataset.csv"
config["train_data_path"] = "./dataset/bart/pretrain/1_dataset/train_data.csv"

config["min_len_token"] = 6
config["max_len_token"] = 96

config["lang_src"] = "noise_vi"
config["lang_tgt"] = "vi"

config["vocab_size_src"] = 100000
config["vocab_size_tgt"] = 100000
config["min_frequency"] = 2
config["special_tokens"] = [
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
    "<cls>",
    "<sep>",
]
config["type_token_src"] = WORDPIECE_TOKEN
config["type_token_tgt"] = WORDPIECE_TOKEN

config["tokenizer_src_path"] = "./dataset/bart/pretrain/1_dataset/tokenizer/tokenizer_src.json"
config["tokenizer_tgt_path"] = "./dataset/bart/pretrain/1_dataset/tokenizer/tokenizer_tgt.json"

config["graph_len_token_src_path"] = "./dataset/bart/pretrain/1_dataset/statistic/len_token_src.png"
config["graph_len_token_tgt_path"] = "./dataset/bart/pretrain/1_dataset/statistic/len_token_tgt.png"

config["desc_path"] = "./dataset/bart/pretrain/1_dataset/desc.txt"

# read raw data

In [4]:
raw_data = pd.read_csv(config["raw_data_path"])[:config["max_sample"]]
raw_data = raw_data.dropna()
raw_data = raw_data.drop_duplicates()
raw_data

,URL,Title,Summary,Contents,Date,Author(s),Category,Tags
0,https://laodong.vn/bat-dong-san/thong-tin-ngoc...,"Thông tin “Ngọc Trinh mua đất ở Bảo Lộc"" chỉ l...","Lâm Đồng - Lãnh đạo thành phố Bảo Lộc, Lâm Đồn...","Những ngày vừa qua, trên trang Facebook chính ...","Thứ sáu, 20/05/2022 08:56 (GMT+7)",Phương Nhiên,Bất động sản,"['Lâm Đồng', 'Ngọc Trinh', 'Chiêu trò', 'Giá đ..."
1,https://laodong.vn/bat-dong-san/lo-hong-trong-...,Lỗ hổng trong việc thẩm tra năng lực tài chính...,TPHCM - Việc không thể cưỡng chế thuế của hai ...,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua...","Thứ sáu, 20/05/2022 08:10 (GMT+7)",Gia Miêu,Bất động sản,"['Thủ Thiêm', 'Đấu giá đất']"
2,https://laodong.vn/bat-dong-san/som-hoan-thien...,Sớm hoàn thiện các dự án nhà ở xã hội để CNLĐ ...,"Hiện trên địa bàn tỉnh Ninh Bình có 32 khu, cụ...",CNLĐ mong muốn sớm được tiếp cận với nhà ở xã ...,"Thứ sáu, 20/05/2022 07:47 (GMT+7)",NGUYỄN TRƯỜNG,Bất động sản,"['Dự án', 'Nhà ở xã hội', 'Dự án nhà ở xã hội'..."
3,https://laodong.vn/bat-dong-san/chi-tiet-ho-so...,Chi tiết hồ sơ hoàn công nhà ở năm 2022,Hoàn công nhà ở với ý nghĩa là điều kiện để đư...,Hoàn công nhà ở là một thủ tục hành chính tron...,"Thứ sáu, 20/05/2022 06:44 (GMT+7)",Kim Nhung (T/H),Bất động sản,"['Giấy phép xây dựng', 'Hồ sơ hoàn công', 'nhà..."
4,https://laodong.vn/bat-dong-san/khoi-tao-khong...,"Khởi tạo không gian sống đẳng cấp, đón sóng đầ...",Có rất nhiều lý do khiến những dự án thấp nội ...,Đi dọc đường Lê Văn Lương kéo dài xuống khu Dư...,"Thứ năm, 19/05/2022 15:30 (GMT+7)",Huyền Nguyễn,Bất động sản,['An Quý Villa']
...,...,...,...,...,...,...,...,...
313303,https://laodong.vn/ho-tro-khac-phuc-thien-tai/...,Quảng Trị: Trao 300 triệu đồng cho người dân b...,"Sáng nay (7.10) tại tỉnh Quảng Trị, đồng chí N...",Đoàn cứu trợ đã trực tiếp về huyện Vĩnh Linh v...,"Thứ hai, 07/10/2013 12:02 (GMT+7)",Hưng Thơ,Tin hoạt động,"['Quảng Trị', 'Bão']"
313304,https://laodong.vn/ho-tro-khac-phuc-thien-tai/...,Trao 100 triệu đồng hỗ trợ CNVCLĐ tỉnh Nghệ An...,"Sáng nay (7.10), tại Nghệ An, Quỹ Tấm lòng vàn...",Đồng chí Phạm Anh Tuấn cũng cho biết: Chiều qu...,"Thứ hai, 07/10/2013 11:53 (GMT+7)",Bảo Duy,Tin hoạt động,"['Nghệ An', 'Bão']"
313306,https://laodong.vn/ho-tro-khac-phuc-thien-tai/...,Quỹ Tấm lòng vàng Lao Động: Trao 4 triệu đồng ...,Đại diện Quỹ Tấm lòng Lao Động và LĐLĐ Phú Yên...,Số tiền trên được trích từ nguồn Quỹ Tấm lòng ...,"Chủ nhật, 06/10/2013 19:33 (GMT+7)",L.Phong,Tin hoạt động,"['Quỹ Tấm lòng Vàng', 'Quỹ Tấm lòng vàng Lao Đ..."
313307,https://laodong.vn/ho-tro-khac-phuc-thien-tai/...,Thêm những tấm lòng thơm thảo hướng về đồng bà...,"Tính đến chiều 5.10, Quỹ Tấm lòng vàng (TLV) L...","Theo đó, Quỹ TLV Lao Động quyết định hỗ trợ 6...","Thứ bảy, 05/10/2013 15:51 (GMT+7)",Bảo Duy,Tin hoạt động,"['Miền Trung', 'Đồng bào miền Trung']"


In [5]:
raw_data = raw_data.rename(columns={"Contents": config["lang_src"]})
raw_data = raw_data[[config["lang_src"]]]
raw_data = raw_data.drop_duplicates()
raw_data = raw_data.dropna()
raw_data

,noise_vi
0,"Những ngày vừa qua, trên trang Facebook chính ..."
1,"Theo thông tin từ Cục Thuế TP.HCM, hiện cơ qua..."
2,CNLĐ mong muốn sớm được tiếp cận với nhà ở xã ...
3,Hoàn công nhà ở là một thủ tục hành chính tron...
4,Đi dọc đường Lê Văn Lương kéo dài xuống khu Dư...
...,...
313303,Đoàn cứu trợ đã trực tiếp về huyện Vĩnh Linh v...
313304,Đồng chí Phạm Anh Tuấn cũng cho biết: Chiều qu...
313306,Số tiền trên được trích từ nguồn Quỹ Tấm lòng ...
313307,"Theo đó, Quỹ TLV Lao Động quyết định hỗ trợ 6..."


# split sentence

In [6]:
raw_data[config["lang_src"]] = raw_data[config["lang_src"]].apply(separate_text)
raw_data = raw_data.explode(config["lang_src"])
raw_data[config["lang_tgt"]] = raw_data[config["lang_src"]]
raw_data.reset_index(drop=True, inplace=True)
raw_data

,noise_vi,vi
0,"Những ngày vừa qua, trên trang Facebook chính ...","Những ngày vừa qua, trên trang Facebook chính ..."
1,"Trước thông tin này, ông Đoàn Kim Đình - Chủ t...","Trước thông tin này, ông Đoàn Kim Đình - Chủ t..."
2,"Qua xác minh, khu đất Ngọc Trinh chụp hình và ...","Qua xác minh, khu đất Ngọc Trinh chụp hình và ..."
3,Nhưng đây không phải là đất xây dựng homestay ...,Nhưng đây không phải là đất xây dựng homestay ...
4,Trong các hồ sơ thủ tục liên quan đất đai trên...,Trong các hồ sơ thủ tục liên quan đất đai trên...
...,...,...
4254261,ĐT: 08.35171402; fax: 08.35171407.,ĐT: 08.35171402; fax: 08.35171407.
4254262,Hoặc chuyển khoản về Quỹ Xã hội từ thiện Tấm l...,Hoặc chuyển khoản về Quỹ Xã hội từ thiện Tấm l...
4254263,"Báo Lao Động, Quỹ TLV Lao Động chân thành cảm ...","Báo Lao Động, Quỹ TLV Lao Động chân thành cảm ..."
4254264,* LĐLĐ TP.,* LĐLĐ TP.


# filter number character

In [7]:
raw_data = raw_data[raw_data.apply(
    lambda text: condition_non_number_character(
        text=text[config["lang_src"]],
    ) and condition_non_number_character(
        text=text[config["lang_tgt"]],
    ),
    axis=1,
)]
raw_data.reset_index(drop=True, inplace=True)
raw_data = raw_data[:int(config["max_get_sample"] * 1.1)]
raw_data

,noise_vi,vi
0,"Trước thông tin này, ông Đoàn Kim Đình - Chủ t...","Trước thông tin này, ông Đoàn Kim Đình - Chủ t..."
1,Nhưng đây không phải là đất xây dựng homestay ...,Nhưng đây không phải là đất xây dựng homestay ...
2,Trong các hồ sơ thủ tục liên quan đất đai trên...,Trong các hồ sơ thủ tục liên quan đất đai trên...
3,Đại diện lãnh đạo UBND TP.Bảo Lộc cũng nói thê...,Đại diện lãnh đạo UBND TP.Bảo Lộc cũng nói thê...
4,"Việc này, khiến dư luận đánh giá sai về công t...","Việc này, khiến dư luận đánh giá sai về công t..."
...,...,...
2199995,"Riêng trường hợp đặc biệt, cấp bách mà cá nhân...","Riêng trường hợp đặc biệt, cấp bách mà cá nhân..."
2199996,"Tuy nhiên, khi trả kết quả thì được trả kết qu...","Tuy nhiên, khi trả kết quả thì được trả kết qu..."
2199997,UBND huyện Tiền Hải yêu cầu Công ty TNHH Hưng ...,UBND huyện Tiền Hải yêu cầu Công ty TNHH Hưng ...
2199998,Nếu Công ty TNHH Hưng Phú Cường không khắc phụ...,Nếu Công ty TNHH Hưng Phú Cường không khắc phụ...


# normalize data

In [8]:
def mappling_item(item):
    return  contraction(normalize_punctuation_spacing(item.lower())).strip()
raw_data[config["lang_src"]] = raw_data[config["lang_src"]].map(lambda item: item.lower())
raw_data[config["lang_tgt"]] = raw_data[config["lang_tgt"]].map(lambda item: item.lower())

raw_data[config["lang_src"]] = raw_data[config["lang_src"]].map(lambda item: normalize_punctuation_spacing(item))
raw_data[config["lang_tgt"]] = raw_data[config["lang_tgt"]].map(lambda item: normalize_punctuation_spacing(item))

raw_data[config["lang_src"]] = raw_data[config["lang_src"]].map(lambda item: contraction(item))
raw_data[config["lang_tgt"]] = raw_data[config["lang_tgt"]].map(lambda item: contraction(item))

raw_data[config["lang_src"]] = raw_data[config["lang_src"]].map(lambda item: item.strip())
raw_data[config["lang_tgt"]] = raw_data[config["lang_tgt"]].map(lambda item: item.strip())
raw_data

# build tokenizer for dataset

In [ ]:
dataset = raw_data[[config["lang_src"], config["lang_tgt"]]]
dataset = dataset.drop_duplicates()
dataset = dataset.dropna()
dataset.reset_index(drop=True, inplace=True)
dataset

In [ ]:
trainer_tokenizer_src = ApiTokenizerHuggingFace(
    dataset=dataset[config["lang_src"]],
    vocab_size=config["vocab_size_src"],
    min_frequency=config["min_frequency"],
    special_tokens=config["special_tokens"],
    type_token=config["type_token_src"],
)

trainer_tokenizer_tgt = ApiTokenizerHuggingFace(
    dataset=dataset[config["lang_tgt"]],
    vocab_size=config["vocab_size_tgt"],
    min_frequency=config["min_frequency"],
    special_tokens=config["special_tokens"],
    type_token=config["type_token_tgt"],
)

In [ ]:
# train tokenizer
tokenzier_src = trainer_tokenizer_src.train()
tokenzier_tgt = trainer_tokenizer_tgt.train()

# save tokenizer
tokenzier_src.save(config["tokenizer_src_path"])
tokenzier_tgt.save(config["tokenizer_tgt_path"])

# read tokenizer
tokenizer_src, tokenizer_tgt = read_tokenizer(
    tokenizer_src_path=config["tokenizer_src_path"],
    tokenizer_tgt_path=config["tokenizer_tgt_path"],
)

config["vocab_size_src"] = tokenzier_src.get_vocab_size()
config["vocab_size_tgt"] = tokenzier_tgt.get_vocab_size()

print("Vocab size src: ", config["vocab_size_src"])
print("Vocab size tgt: ", config["vocab_size_tgt"])

# fillter length tokens

In [ ]:
dataset = dataset[dataset.apply(
    lambda text: condition_length_with_tokenizer(
        tokenizer=tokenizer_src,
        text=text[config["lang_src"]],
        min_len_token=config["min_len_token"],
        max_len_token=config["max_len_token"],
    ) and condition_length_with_tokenizer(
        tokenizer=tokenizer_tgt,
        text=text[config["lang_tgt"]],
        min_len_token=config["min_len_token"],
        max_len_token=config["max_len_token"],
    ),
    axis=1,
)]

dataset = dataset[:config["max_get_sample"]]
dataset.reset_index(drop=True, inplace=True)

# save dataset
dataset.to_csv(config["train_data_path"], index=False)

# read dataset
dataset = pd.read_csv(config["train_data_path"])

In [ ]:
lenght_data_src = get_length_tokens(
    tokenizer=tokenizer_src,
    dataset=dataset[config["lang_src"]],
)

lenght_data_tgt = get_length_tokens(
    tokenizer=tokenizer_tgt,
    dataset=dataset[config["lang_tgt"]],
)

config["min_len_token"] = min(lenght_data_src + lenght_data_tgt)
config["max_len_token"] = max(lenght_data_src + lenght_data_tgt)

draw_hist_graph(
    title="Histogram length tokens",
    xlabel="Length tokens",
    ylabel="Frequency",
    data=lenght_data_src,
    save_path=config["graph_len_token_src_path"],
)
draw_hist_graph(
    title="Histogram length tokens",
    xlabel="Length tokens",
    ylabel="Frequency",
    data=lenght_data_tgt,
    save_path=config["graph_len_token_tgt_path"],
)

# save desc

In [ ]:
config["length_dataset"] = len(dataset)
config["desc"] = f"Vocab size src: {config['vocab_size_src']}\nVocab size tgt: {config['vocab_size_tgt']}\nMin frequency: {config['min_frequency']}\nMin len token: {config['min_len_token']}\nMax len token: {config['max_len_token']}\nType token src: {config['type_token_src']}\nType token tgt: {config['type_token_tgt']}\nSpecial tokens: {config['special_tokens']}\nLength dataset: {config['length_dataset']}\n"
write_file(
    file_name=config["desc_path"],
    content=config["desc"],
)